In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 加载Olivetti Face数据集
train_dataset = datasets.OlivettiFaces(root='./data', download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 定义简单的CNN模型
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 40)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = x.view(-1, 32 * 16 * 16)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# 初始化模型、损失函数和优化器
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()

# 不同优化器的训练过程
optimizers = {
    'SGD': optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001),
    'Adam': optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001),
    'RMSprop': optim.RMSprop(model.parameters(), lr=0.001, alpha=0.99, weight_decay=0.001)
}

# 训练函数
def train_model(optimizer, epochs=10):
    model.train()
    train_losses = []
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        train_losses.append(running_loss / len(train_loader))
        print(f'Epoch {epoch+1}/{epochs}, Loss: {train_losses[-1]:.4f}')
    return train_losses

# 训练并记录loss
losses = {}
for name, optimizer in optimizers.items():
    print(f'Training with {name} optimizer...')
    losses[name] = train_model(optimizer)

# 绘制loss曲线
plt.figure(figsize=(10, 6))
for name, loss in losses.items():
    plt.plot(loss, label=name)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss Comparison')
plt.legend()
plt.show()

AttributeError: module 'torchvision.datasets' has no attribute 'OlivettiFaces'